In [1]:
import tensorflow as tf
import glob

from config import TRANSFORMED_TRAIN_ANNOTATIONS_PATH,IMAGE_SIZE
from models.six_stage_linear_model import ModelMaker
import dataset_functions



In [2]:
import visualizations as v
import numpy as np
import matplotlib.pyplot as plt

---
# Make dataset

In [5]:
tfrecord_files=glob.glob(TRANSFORMED_TRAIN_ANNOTATIONS_PATH+"-*.tfrecords")
ds_raw = tf.data.TFRecordDataset(tfrecord_files)

In [6]:
tfrecord_files

['./dataset/transformed/person_keypoints_train2017-001.tfrecords',
 './dataset/transformed/person_keypoints_train2017-002.tfrecords']

In [7]:
TF_parser=dataset_functions.TFrecordParser()
ds_parsed=ds_raw.map(TF_parser.read_tfrecord)

In [8]:
label_transformer=dataset_functions.LabelTransformer()

@tf.function
def make_label_tensors(elem):
    idd=elem['id']
    kpt_tr=label_transformer.keypoints_spots_vmapfn(elem['kpts'])
    paf_tr=label_transformer.joints_PAFs(elem['joints'])
    
    image_raw=elem["image_raw"]
    image=tf.image.decode_jpeg(image_raw,channels=3)
    image=tf.image.convert_image_dtype(image,dtype=tf.float32)
    image=tf.image.resize(image,IMAGE_SIZE)
    return image,(paf_tr,kpt_tr),idd

In [9]:
@tf.function
def place_training_labels(image,labels,idd):
    paf_tr=labels[0]
    kpt_tr=labels[1]
    return image,(paf_tr,paf_tr,paf_tr,paf_tr,kpt_tr,kpt_tr) #this should match the model outputs, and is different for each model

In [20]:
#ds=parsed_ds.cache()
ds_tnlables=ds_parsed.map(make_label_tensors)
ds_tnlabels_arranged=ds_tnlables.map(place_training_labels)

In [26]:
ds_batched=ds_tnlabels_arranged.batch(32)
#shuffle,repeat,prefetch, etc here
ds=ds_batched.repeat()

---
# Model

In [27]:
#@tf.function
def mse_2d_loss(y_true, y_pred):
    pixel_losses=tf.keras.losses.mean_squared_error(y_true, y_pred)
    return tf.math.reduce_mean(pixel_losses,axis=-1)

In [28]:
model_maker=ModelMaker()
train_model,test_model=model_maker.create_models()

In [33]:
train_model.compile(optimizer=tf.keras.optimizers.Adam(),
                    loss=mse_2d_loss,
                    #metrics=["acc"]
                   )

---
Actually training

In [34]:
train_model.fit(ds,epochs=2,steps_per_epoch=1750)

Train for 1750 steps
Epoch 1/2
 142/1750 [=>............................] - ETA: 58:06 - loss: 2.5903 - stage1paf_output_loss: 7.9626e-04 - stage2paf_output_loss: 7.9631e-04 - stage3paf_output_loss: 8.0051e-04 - stage4paf_output_loss: 0.0010 - stage5heatmap_output_loss: 0.2687 - stage6heatmap_output_loss: 2.3182

KeyboardInterrupt: 

In [31]:
56000/32

1750.0

In [19]:
#v.show_pafs_kpts_img(img.numpy(),paf.numpy(),kpt.numpy(),1,1) #can be used to draw the tensor data